# Zero-Coupon Bond Using Marlowe\'s Command-Line Interface (Liên kết bằng phiếu giảm giá bằng cách sử dụng API REST của Marlowe Runtime)

***Trước khi chạy sổ ghi chép này, bạn có thể muốn sử dụng chức năng "xóa đầu ra" của Jupyter để xóa kết quả thực hiện trước đó của sổ ghi chép này. Điều đó sẽ làm rõ hơn những gì đã được thực thi trong phiên hiện tại.***

Ví dụ về trái phiếu không INTEREST là một hợp đồng Marlowe đơn giản trong đó người cho vay cung cấp tiền gốc cho người đi vay, người này sẽ hoàn trả lại số tiền đó cùng với tiền lãi.

[Một video hoạt động thông qua máy tính xách tay Jupyter này.](https://youtu.be/ELc72BKf7ec)

Bạn có thể đặt câu hỏi về Marlowe trong [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) hoặc đăng các vấn đề với bài học này lên [danh sách các vấn đề của Bộ công cụ dành cho người mới bắt đầu Marlowe kho lưu trữ github](https://github.com/input-output-hk/marlowe-starter-kit/issues).

Trong phần trình diễn này, chúng ta sử dụng API REST của Marlowe Runtime\, được cung cấp qua, `marlowe-cli`, để chạy hợp đồng này trên mạng thử nghiệm công khai `preprod` của Cardano\. Hợp đồng Marlowe có thể sử dụng địa chỉ hoặc vai trò token để ủy quyền: ở đây chúng ta sử dụng vai trò token và chúng ta có Marlowe Runtime đúc chúng.

Trong [Marlowe Playground](https://play.marlowe-finance.io/), hợp đồng có dạng như sau ở định dạng Blockly.

![Zero-coupon bond Marlowe contract](images/01-zcb-contract.png)

Ở định dạng Marlowe, nó xuất hiện dưới dạng
```
When
    [Case
        (Deposit
            (Address "$LENDER_ADDR")
            (Address "$LENDER_ADDR")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Address "$LENDER_ADDR")
            (Party (Address "$BORROWER_ADDR"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Address "$BORROWER_ADDR")
                        (Address "$BORROWER_ADDR")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Address "$BORROWER_ADDR")
                        (Party (Address "$LENDER_ADDR"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close 
```

## Chuẩn bị

Xem [Bài 0. Chuẩn bị](00-preliminaries.md) để biết thông tin về cách thiết lập môi trường của một người để sử dụng hướng dẫn này.

Bài học giả định rằng các biến môi trường sau đây đã được thiết lập.
- `CARDANO_NODE_SOCKET_PATH`: vị trí socket của node Cardano.
- `CARDANO_TESTNET_MAGIC`: số magic testnet.

Nó cũng giả định rằng các bên Bên cho vay và Bên vay có địa chỉ, khóa ký và tiền.
- Người cho vay
    - [keys/lender.address](keys/lender.address): Địa chỉ Cardano của người cho vay
    - [keys/lender.skey](keys/lender.skey): vị trí ký tệp khóa cho bên cho vay
- Người cho vay
    - [keys/borrower.address](keys/borrower.address): Địa chỉ Cardano cho người vay
    - [keys/borrower.skey](keys/borrower.skey): vị trí ký file key cho bên vay

### Truy cập vào node Cardano và Runtimes Marlowe​

Nếu chúng ta đang sử dụng tiện ích mở rộng Cardano Marlowe Runtime của [demeter.run](https://demeter.run/), thì chúng ta đã có quyền truy cập vào Cardano Node và Marlowe Runtime. Các lệnh sau sẽ đặt các biến môi trường cần thiết để sử dụng triển khai docker cục bộ trên các cổng mặc định. Nó cũng sẽ thiết lập một số biến môi trường bổ sung.

In [1]:
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

CARDANO_NODE_SOCKET_PATH = ~/.local/share/containers/storage/volumes/marlowe-starter-kit_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1


Lưu ý số magic mạng thử nghiệm:
- `preprod` = 1
- `preview` = 2

### Địa chỉ và tiền của người cho vay​

Kiểm tra xem địa chỉ và khóa đã được tạo cho người cho vay chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong [Lesson 0. Preliminaries](00-preliminaries.md).

In [2]:
LENDER_SKEY=keys/lender.skey
LENDER_ADDR=$(cat keys/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Kiểm tra xem người cho vay có ít nhất một trăm ADA không.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ffb18377900e003285f9cf357f96b1691eac3e32a89916e6c49a35bd9a11cdf2     1        1000000000 lovelace + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [4]:
echo "$EXPLORER_URL"/address/"$LENDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Địa chỉ và tiền của người vay

Kiểm tra xem địa chỉ và khóa đã được tạo cho người mượn chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_ADDR=$(cat keys/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Kiểm tra xem người vay có ít nhất một trăm ADA không.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ffb18377900e003285f9cf357f96b1691eac3e32a89916e6c49a35bd9a11cdf2     2        1000000000 lovelace + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [7]:
echo "$EXPLORER_URL"/address/"$BORROWER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Thiết kế hợp đồng

Bạn có thể tải xuống hợp đồng trái phiếu không nhận phiếu thưởng từ [Marlowe Playground](https://play.marlowe-finance.io/) dưới dạng tệp JSON hoặc có thể tạo bằng cách sử dụng Marlowe CLI [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) bằng cách sử dụng lệnh `marlowe-cli template`.

Đặt tiền gốc của khoản vay là 80 ADA và INTEREST là 5 ADA.

In [8]:
ADA=1000000  # 1 ada = 1,000,000 lovelace

PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))

echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


Trên blockchain Cardano, các tham số giao thức yêu cầu mỗi UTxO chứa ít nhất một số ADA. Ở đây chúng ta sẽ bắt đầu hợp đồng với 2 ADA.

In [9]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Ở phần sau của ví dụ, chúng ta sẽ cần một số hằng số để chuyển đổi thời gian.

In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

NOW = 1679661079000 POSIX milliseconds = Fri Mar 24 06:31:19 AM MDT 2023


Hợp đồng có thời hạn cho vay và thời hạn trả nợ. Để thuận tiện trong ví dụ này, hãy đặt thời hạn cho tương lai gần.

In [11]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo LENDER_DEADLINE = "$LENDER_DEADLINE" POSIX milliseconds = "`date -d @$((LENDER_DEADLINE / SECOND))`"
echo BORROWER_DEADLINE = "$BORROWER_DEADLINE" POSIX milliseconds = "`date -d @$((BORROWER_DEADLINE / SECOND))`"

LENDER_DEADLINE = 1679664679000 POSIX milliseconds = Fri Mar 24 07:31:19 AM MDT 2023
BORROWER_DEADLINE = 1679671879000 POSIX milliseconds = Fri Mar 24 09:31:19 AM MDT 2023


Bây giờ hãy tạo tệp JSON cho hợp đồng, `zcb-contract.json`.

In [12]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file zcb-state.json

Các tùy chọn dòng lệnh khác nhau được mô tả bởi hệ thống trợ giúp.

In [13]:
marlowe-cli template zcb --help

Usage: marlowe-cli template zcb --minimum-ada INTEGER --lender PARTY
                                --borrower PARTY --principal INTEGER
                                --interest INTEGER --lending-deadline TIMEOUT
                                --repayment-deadline TIMEOUT

  Create a zero-coupon bond.

Available options:
  --minimum-ada INTEGER    Lovelace that the lender contributes to the initial
                           state.
  --lender PARTY           The lender.
  --borrower PARTY         The borrower.
  --principal INTEGER      The principal, in lovelace.
  --interest INTEGER       The interest, in lovelace.
  --lending-deadline TIMEOUT
                           The lending deadline. POSIX milliseconds or duration:
                           `INTEGER[s|m|d|w|h]`.
  --repayment-deadline TIMEOUT
                           The repayment deadline. POSIX milliseconds or
                           duration: `INTEGER[s|m|d|w|h]`.
  -h,--help                Show this help text


## Kiểm tra hợp đồng

Xem tệp hợp đồng dưới dạng YAML.

In [14]:
json2yaml zcb-contract.json

timeout: 1679664679000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
  then:
    from_account:
      address: addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
    pay: 80000000
    then:
      timeout: 1679671879000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            address: addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
        then:
          from_account:
            address: addr_test1vpy4n4peh4su

### \[Không bắt buộc, nhưng nên dùng\] Kiểm tra độ an toàn của hợp đồng

Nếu chúng ta đang chạy hợp đồng trên Cardano `mainnet`, thì chúng ta muốn kiểm tra tính an toàn của nó trước khi tạo nó, để không có khả năng chúng ta có thể mất tiền.

Dưới đây là các bước để kiểm tra mức độ an toàn của hợp đồng:

1. Hiểu [Ngôn ngữ Marlowe](https://marlowe-finance.io/).
2. Hiểu [Mô hình UTxO mở rộng](https://docs.cardano.org/learn/eutxo-explainer) của Cardano.
3. Đọc và hiểu [Hướng dẫn về các phương pháp hay nhất của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Đọc và hiểu [Hướng dẫn bảo mật của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Sử dụng [Marlowe Playground](https://play.marlowe-finance.io/) để gắn cờ cảnh báo, thực hiện phân tích tĩnh và mô phỏng hợp đồng.
6. Sử dụng [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` công cụ để nghiên cứu xem hợp đồng có thể chạy trên mạng Cardano hay không.
7. Chạy *tất cả đường dẫn thực thi* của hợp đồng trên [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Xem [Bài 1](01-runtime-cli.ipynb) để biết ví dụ về cách thực hiện bước 6.

## Giao dịch 1. Tạo Hợp đồng

Lệnh của Marlowe CLI\ `marlowe-cli run initialize` sẽ tạo thông tin tạo cho hợp đồng Marlowe. Chúng ta cung cấp cho nó các tệp JSON chứa hợp đồng và trạng thái ban đầu. Bất kỳ ai cũng có thể tạo hợp đồng, nhưng trong ví dụ này, người cho vay sẽ làm như vậy, vì vậy chúng ta cung cấp địa chỉ của họ để tài trợ cho giao dịch và nhận tiền thay đổi từ đó.

In [15]:
marlowe-cli run initialize --help

Usage: marlowe-cli run initialize [--testnet-magic INTEGER] 
                                  [--socket-path SOCKET_FILE] 
                                  [--stake-address ADDRESS] 
                                  [--roles-currency CURRENCY_SYMBOL]
                                  --contract-file CONTRACT_FILE
                                  --state-file STATE_FILE 
                                  [--at-address ADDRESS | 
                                    --permanently STAKING_ADDRESS | 
                                    --permanently-without-staking] 
                                  [--out-file OUTPUT_FILE] [--merkleize] 
                                  [--print-stats]

  Initialize the first transaction of a Marlowe contract and write output to a
  JSON file.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                         

In [16]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file zcb-contract.json \
  --state-file zcb-state.json \
  --out-file marlowe-1.json

Bây giờ, chúng ta sử dụng lệnh `marlowe-cli run auto-execute` của Marlowe CLI\ để xây dựng và gửi giao dịch tạo.



In [17]:
marlowe-cli run auto-execute --help

Usage: marlowe-cli run auto-execute 
         [--testnet-magic INTEGER] [--socket-path SOCKET_FILE] 
         [--marlowe-in-file MARLOWE_FILE --tx-in-marlowe TXID#TXIX]
         --marlowe-out-file MARLOWE_FILE --change-address ADDRESS
         (--required-signer SIGNING_FILE) [--metadata-file METADATA_FILE]
         --out-file FILE [--submit SECONDS] [--print-stats] [--script-invalid]

  [EXPERIMENTAL] Run a Marlowe transaction, selecting transaction inputs and
  outputs automatically.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-in-file MARLOWE_FILE
                           JSON file with the Marlowe initial state and initial
                 

In [18]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file marlowe-1.json \
  --change-address "$LENDER_ADDR" \
  --required-signer "$LENDER_SKEY" \
  --out-file tx-1.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 212185
Size: 941 / 16384 = 5%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = f36feb37473ec171d9a79553b9a7f729b78076e97be20f54aaca88094a1752d5


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [19]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/f36feb37473ec171d9a79553b9a7f729b78076e97be20f54aaca88094a1752d5?tab=utxo


Người ta cũng có thể kiểm tra UTxO của hợp đồng bằng cách sử dụng `cardano-cli`.

In [20]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f36feb37473ec171d9a79553b9a7f729b78076e97be20f54aaca88094a1752d5     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "c4bf1737c9a1f88ae66c9d00d66d54d201c2c932ab13767889758ba28c8f5f19"


## Giao dịch 2. Người cho vay gửi tiền gốc

Người cho vay gửi 80 ADA tiền gốc của họ vào hợp đồng bằng cách sử dụng lệnh `marlowe-cli run` chuẩn bị của Marlowe CLI\. Người cho vay đang cung cấp vốn cho và nhận tiền thay đổi từ giao dịch này, vì vậy chúng ta cung cấp địa chỉ của họ.

In [21]:
marlowe-cli run prepare --help

Usage: marlowe-cli run prepare --marlowe-file MARLOWE_FILE 
                               [--deposit-account PARTY --deposit-party PARTY 
                                 [--deposit-token TOKEN]
                                 --deposit-amount INTEGER |
                                 --choice-name NAME --choice-party PARTY
                                 --choice-number INTEGER |
                                 --notify] --invalid-before POSIX_TIME
                               --invalid-hereafter POSIX_TIME 
                               [--out-file OUTPUT_FILE] [--print-stats]

  Prepare the next step of a Marlowe contract and write the output to a JSON
  file.

Available options:
  --marlowe-file MARLOWE_FILE
                           JSON input file for the Marlowe state and contract.
  --deposit-account PARTY  The account for the deposit.
  --deposit-party PARTY    The party making the deposit.
  --deposit-token TOKEN    The token being deposited, if not Ada.
  --deposit-

In [22]:
marlowe-cli run prepare \
  --deposit-account "$LENDER_ADDR" \
  --deposit-party "$LENDER_ADDR" \
  --deposit-amount "$PRINCIPAL" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-1.json \
  --out-file marlowe-2.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1679661371000},POSIXTime {getPOSIXTime = 1679661731999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1679661371000},POSIXTime {getPOSIXTime = 1679661731999}), txInputs = [NormalInput (IDeposit "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\"" "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\"" (Token "" "") 80000000)]}
Payment 1
  Acccount: "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\""
  Payee: Party "\"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d\""
  Ada: Lovelace {getLovelace = 80000000}


Một lần nữa, hãy sử dụng `marlowe-cli run auto-execute` để tạo và gửi giao dịch rồi đợi xác nhận.

In [23]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file marlowe-1.json \
  --marlowe-out-file marlowe-2.json \
  --change-address "$LENDER_ADDR" \
  --required-signer "$LENDER_SKEY" \
  --out-file tx-2.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 758511
Size: 1570 / 16384 = 9%
Execution units:
  Memory: 6730260 / 14000000 = 48%
  Steps: 1806177146 / 10000000000 = 18%
TX_2 = 017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [24]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02?tab=utxo


Người ta có thể thấy rằng người cho vay có ít hơn 82 ADA so với ban đầu. Hai ADA đã được gửi vào hợp đồng khi nó được tạo ra và 80 ADA đã được trả cho người vay trong giao dịch thứ hai.

In [25]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02     0        917029304 lovelace + TxOutDatumNone


Người vay hiện có thêm 80 ADA (tiền gốc của khoản vay).

In [26]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02     2        80000000 lovelace + TxOutDatumNone
ffb18377900e003285f9cf357f96b1691eac3e32a89916e6c49a35bd9a11cdf2     2        1000000000 lovelace + TxOutDatumNone


Hợp đồng Marlowe vẫn có 2 ADA từ khi tạo ra nó.

In [27]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "49f4b85150d30d1b9a1cd06dacf45dbabcf0719a34f5286a03c7c52760880c2e"


## Giao dịch 3. Người vay hoàn trả khoản vay

Sau một thời gian, người vay trả lại tiền gốc cộng với tiền lãi. Vì vậy, họ tài trợ cho giao dịch và nhận tiền thay đổi tại địa chỉ của họ.

In [28]:
marlowe-cli run prepare \
  --deposit-account "$BORROWER_ADDR" \
  --deposit-party "$BORROWER_ADDR" \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-2.json \
  --out-file marlowe-3.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1679661602000},POSIXTime {getPOSIXTime = 1679661962999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1679661602000},POSIXTime {getPOSIXTime = 1679661962999}), txInputs = [NormalInput (IDeposit "\"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d\"" "\"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d\"" (Token "" "") 85000000)]}
Payment 1
  Acccount: "\"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d\""
  Payee: Party "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\""
  Ada: Lovelace {getLovelace = 85000000}
Payment 2
  Acccount: "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\""
  Payee: Party "\"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck\""
  Ada: Lovelace {getLovelace = 2000000}


Một lần nữa, hãy sử dụng `marlowe-cli run auto-execute` để tạo và gửi giao dịch rồi đợi xác nhận.

In [29]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file marlowe-2.json \
  --marlowe-out-file marlowe-3.json \
  --change-address "$BORROWER_ADDR" \
  --required-signer "$BORROWER_SKEY" \
  --out-file tx-3.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 569736
Size: 827 / 16384 = 5%
Execution units:
  Memory: 4674622 / 14000000 = 33%
  Steps: 1258378941 / 10000000000 = 12%
TX_3 = b2486f82eaccef641a3b44de4c07146304238b25b0678ab1bfb70e3bc244c338


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [30]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/b2486f82eaccef641a3b44de4c07146304238b25b0678ab1bfb70e3bc244c338?tab=utxo


Người ta có thể thấy rằng người cho vay đã nhận lại 80 ADA tiền gốc và 2 ADA tiền gửi khi hợp đồng được tạo, cùng với 5 ADA tiền lãi bổ sung, tổng cộng là 87 ADA.

In [31]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02     0        917029304 lovelace + TxOutDatumNone
b2486f82eaccef641a3b44de4c07146304238b25b0678ab1bfb70e3bc244c338     1        87000000 lovelace + TxOutDatumNone


Người vay bây giờ có ít hơn khoảng 5 ADA (tiền lãi của khoản vay) so với ban đầu.

In [32]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
017c068bf76aba6ef57b6b5aab592cd9d861c4903e100794ca7451d16c701a02     2        80000000 lovelace + TxOutDatumNone
b2486f82eaccef641a3b44de4c07146304238b25b0678ab1bfb70e3bc244c338     0        914430264 lovelace + TxOutDatumNone


Hợp đồng Marlowe đã đóng, vì vậy không có đầu ra nào cho địa chỉ tập lệnh của nó.